In [9]:
import numpy as np
import gensim
from gensim.models import word2vec
import logging
#load pretrain vec
model=gensim.models.KeyedVectors.load_word2vec_format('/media/kwane/3160053c-937e-4de9-a540-b28bd2802040/kwane/NLP/data/word2vec.txt',binary=False) #GloVe Model

In [10]:
import numpy as np
path='./obj/'

subjective_examples = list(open(path+"quote.tok.gt9.5000", "r").readlines())
subjective_examples = [s.strip() for s in subjective_examples]
print len(subjective_examples)
objective_examples = list(open(path+"plot.tok.gt9.5000", "r").readlines())
objective_examples = [s.strip() for s in objective_examples]
print len(objective_examples)
max_len=0
for i,line in enumerate(subjective_examples+objective_examples):

    line=str.lower(line)
    words = line.strip().split()
    #print words 
    if len(words)>max_len:
        max_len=len(words)

print max_len


5000
5000
120


In [13]:
#corpus
import torch
import logging
import argparse
import os
import numpy as np
from const import *

word2vec=np.random.uniform(-0.25,0.25,(3,4))

def word2idx(sents, word2idx):
    return [[word2idx[w] if w in word2idx else UNK for w in s] for s in sents]


class Dictionary(object):
    def __init__(self, word2idx={}, idx_num=0):
        self.word2idx = word2idx
        self.idx = idx_num

    def _add(self, word):
        if self.word2idx.get(word) is None:
            self.word2idx[word] = self.idx
            self.idx += 1

    def _convert(self):
        self.idx2word = {v: k for k, v in self.word2idx.items()}

    def __len__(self):
        return self.idx

    def __str__(self):
        return "%s(size = %d)".format(self.__class__.__name__, len(self.idx))


class Words(Dictionary):
    def __init__(self):
        word2idx = {
            WORD[PAD]: PAD,
            WORD[UNK]: UNK
        }
        super(Words,self).__init__(word2idx=word2idx, idx_num=len(word2idx))

    def __call__(self, sents):
        words = set([word for sent in sents for word in sent])
        for word in words:
            self._add(word)


class Labels(Dictionary):
    def __init__(self):
        super(Labels,self).__init__()

    def __call__(self, labels):
        _labels = set(labels)
        for label in _labels:
            self._add(label)


class Corpus(object):
    def __init__(self, pos, neg, save_data, max_len):
        self.pos = os.path.join(pos)
        self.neg = os.path.join(neg)
        self._save_data = save_data

        self.w = Words()
        self.l = Labels()
        self.max_len = max_len

    def parse_data(self, pos, neg):
        """
        fine_grained: Whether to use the fine-grained (50-class) version of TREC
                or the coarse grained (6-class) version.
        """
        _sents, _labels = [], []
        #pos
        for sentence in open(pos):
            sentence=str.lower(sentence)
            words = sentence.strip().split()
            
            if len(words) > self.max_len:
                words = words[:self.max_len]

            _sents += [words]
            _labels.append(1)
        #neg
        for sentence in open(neg):
            sentence=str.lower(sentence)
            words = sentence.strip().split()
            
            if len(words) > self.max_len:
                words = words[:self.max_len]

            _sents += [words]
            _labels.append(0)
        
        self.w(_sents)    
        self._sents = _sents   
        self._labels = _labels
    def _shuffle(self,src,_labels):
        self.indices = np.arange(len(_labels))
        np.random.shuffle(self.indices)
        #print '_shuffle'
        self.src = self.src[self.indices]
        self._labels = self._labels[self.indices]

    def save(self):
        self.parse_data(self.pos,self.neg)
        self.src=word2idx(self._sents, self.w.word2idx)
        self.src=np.asarray(self.src)
        self._labels=np.asarray(self._labels)
        
        self._shuffle(self.src,self._labels)
        #print len(self.src)
        train_size=int(len(self.src)*0.8)
        #valid_size=len(self.src)-train_size
        self.train_src,self.train_labels=self.src[:train_size],self._labels[:train_size]
        self.valid_src,self.valid_labels=self.src[train_size:],self._labels[train_size:]
        print len(self.src),len(self.train_src),len(self.valid_src)
        def mask_(i):#mask for rnn
            mask=np.zeros(self.max_len, dtype=int)
            mask[i-1]=1
            return mask
        def mask_all(i):#mask for bi-rnn & cnn
            mask=np.zeros(self.max_len, dtype=int)
            mask[:i]=np.ones(i, dtype=int)
            return mask
        train_mask=[mask_(len(s)) for s in self.train_src]
        valid_mask=[mask_(len(s)) for s in self.valid_src]
        train_mask,valid_mask=np.asarray(train_mask),np.asarray(valid_mask)
        
        train_mask_all=[mask_all(len(s)) for s in self.train_src]
        valid_mask_all=[mask_all(len(s)) for s in self.valid_src]
        train_mask_all,valid_mask_all=np.asarray(train_mask_all),np.asarray(valid_mask_all)
        #print train_mask
        data = {
            'max_len': self.max_len,
            'dict': {
                'train': self.w.word2idx,
                'vocab_size': len(self.w),
                'label_size': 2,
                'indices':self.indices,
            },
            'train': {
                'src': self.train_src,
                'label': self.train_labels,
                'train_mask':train_mask,
                'train_mask_all':train_mask_all

            },
            'valid': {
                'src': self.valid_src,
                'label': self.valid_labels,
                'valid_mask':valid_mask,
                'valid_mask_all':valid_mask_all

            }
        }

        torch.save(data, self._save_data)
        print('Finish dumping the data to file - [{}]'.format(self._save_data))
        print('words length - [{}]'.format(len(self.w)))
        print('label size - [{}]'.format(len(self.l)))


if __name__ == "__main__":

    
    corpus = Corpus(path+"quote.tok.gt9.5000",path+"plot.tok.gt9.5000", path+"corpus.pt", max_len)
    corpus.save()


10000 8000 2000
Finish dumping the data to file - [./data/obj/corpus.pt]
words length - [23928]
label size - [0]


In [14]:
subjective_examples+objective_examples
data = torch.load(path+'corpus.pt')
vocab_size=data['dict']['vocab_size']
train_dic_num=data['train']['src']
valid_dic_num=data['valid']['src']
dic_num=np.concatenate((train_dic_num,valid_dic_num))
indices=data['dict']['indices']
#print np.array(train_dic_num[0])[1]
#print type(train_dic_num),type(train_dic_num[0])
word2vec=np.zeros((vocab_size,300))
#train
for i,line in enumerate(subjective_examples):
    line=str.lower(line)
    words = line.strip().split()
    words = words[:max_len]
    for j,word in enumerate(words):
        #print i,j,train_dic_num[i][j]
        try:
            word2vec[np.array(dic_num[int(np.where(indices==i)[0])])[j],:]=model[word]
        except:
            word2vec[np.array(dic_num[int(np.where(indices==i)[0])])[j],:]=np.random.uniform(-0.25,0.25,300)
#valid
for i,line in enumerate(objective_examples):
    line=str.lower(line)
    words = line.strip().split()
    words = words[:max_len]
    for j,word in enumerate(words):
        try:
            word2vec[np.array(dic_num[int(np.where(indices==len(subjective_examples)+i)[0])])[j],:]=model[word]
        except:
            word2vec[np.array(dic_num[int(np.where(indices==len(subjective_examples)+i)[0])])[j],:]=np.random.uniform(-0.25,0.25,300)
print train_dic_num[0],train_dic_num[1]
np.save(path+'pretrained_wordvec',word2vec)
#x=np.load('./data/pretrained_wordvec.npy')
#print np.size(word2vec),word2vec

[9169, 15302, 12716, 17976, 6975, 11205, 10875, 17516, 21501, 4097, 6209, 3816, 1166, 246] [7659, 5861, 12716, 7659, 20483, 12552, 8663, 9779, 11210, 7659, 12338, 21999, 21036, 17818, 13776, 6762, 248, 4700, 16422, 19737, 20269, 20055, 626, 246]
